In [31]:
# Preprocess the test data

import pickle
# load the test data
apple_bars_test_data = pickle.load(open('apple_bar_TEST_data.pkl', 'rb'))
#preprocess the test data
#convert the dataframe index to a column
apple_bars_test_data['DateTime'] = apple_bars_test_data.index.to_pydatetime()
# remove the index from the dataframe
apple_bars_df = apple_bars_test_data.reset_index(drop=True)
# show the head of the dataframe
#apple_bars_df.head()
#%%script false
#drop all columns except for the DateTime and the vwap columns
apple_bars_df = apple_bars_df.drop(columns=['open', 'high', 'low', 'close', 'volume','trade_count'])
# show the head of the dataframe
#apple_bars_df.head()
#%%script false
# create a new dataframe with a column called 'Date' that contains the date of the data and colums for every DateTime with the corresponding vwap value

#create an array of all the unique dates in the dataframe
dates = apple_bars_df['DateTime'].dt.date.unique()
# make dates to list
dates = dates.tolist()
dates = list(map(lambda x: x.strftime('%Y-%m-%d'), dates))
#dates

#%%script false
# import pandas as pd if not imported
if not 'pd' in globals():
    import pandas as pd
    
# in a new dataframe, for each date in the array, create a columns for every vwap value for that date
apple_df = pd.DataFrame(index=dates, columns=range(0, 1000))

# get all vwap values for each date
for date in dates:
    # get the vwap values for that date
    vwap = apple_bars_df[apple_bars_df['DateTime'].dt.strftime('%Y-%m-%d') == date]['vwap'].tolist()
    # fill vwap list with zeros to match the length of the columns
    vwap = vwap + [0] * (1000 - len(vwap))
    # create a column for that date
    apple_df.loc[date] = vwap
#save the dataframe to a pickle file
apple_df.to_pickle('apple_TEST_df.pkl')
#load the pickle file
apple_df = pd.read_pickle('apple_TEST_df.pkl')
#import numpy as np if not imported
if not 'np' in globals():
    import numpy as np
# set 'gain' column to 1 if column '0' is less than the last non-zero column in the row
apple_df['gain'] = apple_df.apply(lambda x: True if x[0] < x[x[x>0].last_valid_index()] else False, axis=1)

#print the head of the dataframe showing the index and the gain column as well as column 0 and the last non-zero column
#print(apple_df.head(50)['gain'])
# pickle the dataframe
apple_df.to_pickle('apple_TEST_df_ML.pkl')
#imports
import pickle
import pandas as pd
import numpy as np

#load apple_df_ML.pkl
apple_df_ML = pickle.load(open('apple_TEST_df_ML.pkl', 'rb'))

#drop index column
apple_df_ML.reset_index(drop=True, inplace=True)
apple_df_ML
# function drops any rows with more than 40% 0 values
def drop_rows_with_many_zeros(df):
    counts =(df==0).astype(int).sum(axis=1)
    df = df[counts < len(df.columns)*0.4]
    return df

# function that replaces any nan values with the last non-nan value
def replace_nan_with_last_non_nan(df):
    for row_indx in range(len(df)):
        last_non_nan_value = df.iloc[row_indx,0]
        for col_indx, val in enumerate(df.iloc[row_indx,:]):
            if not pd.notna(val):
                df.iloc[row_indx,col_indx] = last_non_nan_value
            else:
                last_non_nan_value = val
    return df
#only keep values that are not equal to 0 except for the 'gain' column
nan_value = float('NaN')

#replace all 0s with NaN
apple_df_ML.replace(0, nan_value, inplace=True)

#drop rows with more than 40% 0 values
apple_df_ML = drop_rows_with_many_zeros(apple_df_ML)

#replace nan values with the last non-nan value
apple_df_ML = replace_nan_with_last_non_nan(apple_df_ML)


# for the 'gain' column, replace all True with 1 and all False with 0
apple_df_ML.replace(True, 1, inplace=True)
apple_df_ML.replace(False, 0, inplace=True)

# remove all columns between 360 and 999
apple_df_ML.drop(apple_df_ML.columns[360:1000], axis=1, inplace=True)
# save the dataframe to a pickle file
apple_df_ML.to_pickle('apple_df_TEST_ML.pkl')

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras import layers
from tensorflow.keras import activations

In [33]:
#load the model
model = tf.keras.models.load_model('models/apple_model_ML.h5')
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 1000)              361000    
                                                                 
 dropout_31 (Dropout)        (None, 1000)              0         
                                                                 
 dense_47 (Dense)            (None, 750)               750750    
                                                                 
 dropout_32 (Dropout)        (None, 750)               0         
                                                                 
 dense_48 (Dense)            (None, 1)                 751       
                                                                 
Total params: 1,112,501
Trainable params: 1,112,501
Non-trainable params: 0
_________________________________________________________________


In [37]:
#load the dataframe
test_data = pd.read_pickle('apple_df_TEST_ML.pkl')


In [40]:
# use the model to predict the gain column

#drop the gain column
input_data = test_data.drop(columns=['gain'], axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(input_data)
input_data = scaler.transform(input_data)

#predict the gain column
predictions = model.predict(input_data )
predictions[predictions > (predictions.mean())] = 1
predictions[predictions <= (predictions.mean())] = 0
predictions

1/1 [==============================] - 0s 15ms/step


array([[1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [41]:
#import confusion matrix
import sklearn.metrics as metrics

metrics.confusion_matrix(test_data['gain'], predictions)

array([[5, 1],
       [4, 6]])